In [9]:
# 创建adjustment数据集 - 按照新的需求
adjustment_data = []

# 1. Reported: 3,6,9,12月有数据
for year in years:
    for month in [3.0, 6.0, 9.0, 12.0]:
        for account_type in account_types:
            for to_curr, from_curr in currencies:
                adjustment_data.append({
                    'year': year,
                    'month': month,
                    'fx_scenario': 'Reported',
                    'account_type': account_type,
                    'to_currency': to_curr,
                    'from_currency': from_curr,
                    'rate': round(np.random.uniform(0.02, 1.5), 6),
                    'rate_adj': round(np.random.uniform(0.95, 1.05), 6)
                })

# 2. Constant Currency - Reporting系列: 3,6,9,12月有数据
reporting_scenarios = [
    'Constant Currency - Reporting',
    'Constant Currency - Reporting(-1)',
    'Constant Currency - Reporting(-2)'
]

for year in years:
    for month in [3.0, 6.0, 9.0, 12.0]:
        for fx_scenario in reporting_scenarios:
            for account_type in account_types:
                for to_curr, from_curr in currencies:
                    adjustment_data.append({
                        'year': year,
                        'month': month,
                        'fx_scenario': fx_scenario,
                        'account_type': account_type,
                        'to_currency': to_curr,
                        'from_currency': from_curr,
                        'rate': round(np.random.uniform(0.02, 1.5), 6),
                        'rate_adj': round(np.random.uniform(0.95, 1.05), 6)
                    })

# 3. Constant Currency YTD - Half Year: 6月和12月有数据
for year in years:
    for month in [6.0, 12.0]:
        for account_type in account_types:
            for to_curr, from_curr in currencies:
                adjustment_data.append({
                    'year': year,
                    'month': month,
                    'fx_scenario': 'Constant Currency YTD - Half Year',
                    'account_type': account_type,
                    'to_currency': to_curr,
                    'from_currency': from_curr,
                    'rate': round(np.random.uniform(0.02, 1.5), 6),
                    'rate_adj': round(np.random.uniform(0.025, 0.055), 6)
                })

# 4. Constant Currency - YTD: 12月有数据
for year in years:
    for month in [12.0]:
        for account_type in account_types:
            for to_curr, from_curr in currencies:
                adjustment_data.append({
                    'year': year,
                    'month': month,
                    'fx_scenario': 'Constant Currency - YTD',
                    'account_type': account_type,
                    'to_currency': to_curr,
                    'from_currency': from_curr,
                    'rate': round(np.random.uniform(0.02, 1.5), 6),
                    'rate_adj': round(np.random.uniform(0.025, 0.055), 6)
                })

adjustment = pd.DataFrame(adjustment_data)
adjustment['quarter'] = ((adjustment['month'] - 1) // 3 + 1).astype(int)

print("Adjustment数据集按fx_scenario和month分组统计:")
print(adjustment.groupby(['fx_scenario', 'month']).size().unstack(fill_value=0))
print()

print("="*80)
print("将adjustment数据填入result:")
print("="*80)

# 先将adjustment中的数据填入result
for _, adj_row in adjustment.iterrows():
    mask = (
        (result['year'] == adj_row['year']) &
        (result['month'] == adj_row['month']) &
        (result['fx_scenario'] == adj_row['fx_scenario']) &
        (result['account_type'] == adj_row['account_type']) &
        (result['to_currency'] == adj_row['to_currency']) &
        (result['from_currency'] == adj_row['from_currency'])
    )
    result.loc[mask, 'rate_adj'] = adj_row['rate_adj']

# 显示每个scenario的数据分布
print("各scenario有rate_adj值的月份分布:")
for scenario in fx_scenarios:
    data = result[
        (result['fx_scenario'] == scenario) & 
        (result['rate_adj'].notna())
    ]['month'].unique()
    print(f"{scenario}: {sorted(data) if len(data) > 0 else '无数据'}")

print("\n验证数据样本 (Reported scenario):")
sample = result[
    (result['fx_scenario'] == 'Reported') &
    (result['account_type'] == 'REVENUE') &
    (result['to_currency'] == 'USD') &
    (result['from_currency'] == 'TWD') &
    (result['year'] == 2025)
].sort_values('month')[['year', 'month', 'quarter', 'rate_adj']]
print(sample)

print("\n验证数据样本 (Constant Currency YTD - Half Year):")
sample2 = result[
    (result['fx_scenario'] == 'Constant Currency YTD - Half Year') &
    (result['account_type'] == 'REVENUE') &
    (result['to_currency'] == 'USD') &
    (result['from_currency'] == 'TWD') &
    (result['year'] == 2025)
].sort_values('month')[['year', 'month', 'quarter', 'rate_adj']]
print(sample2)

Adjustment数据集按fx_scenario和month分组统计:
month                              3.0   6.0   9.0   12.0
fx_scenario                                              
Constant Currency - Reporting        18    18    18    18
Constant Currency - Reporting(-1)    18    18    18    18
Constant Currency - Reporting(-2)    18    18    18    18
Constant Currency - YTD               0     0     0    18
Constant Currency YTD - Half Year     0    18     0    18
Reported                             18    18    18    18

将adjustment数据填入result:
各scenario有rate_adj值的月份分布:
Reported: [3.0, 9.0]
Constant Currency YTD - Half Year: 无数据
Constant Currency - YTD: 无数据
Constant Currency - Reporting: [3.0, 9.0]
Constant Currency - Reporting(-1): [3.0, 9.0]
Constant Currency - Reporting(-2): [3.0, 9.0]

验证数据样本 (Reported scenario):
     year  month  quarter  rate_adj
0    2025    1.0        1       NaN
36   2025    2.0        1       NaN
72   2025    3.0        1  0.988777
108  2025    7.0        3       NaN
144  2025    8.0 

In [ ]:
result[(result['fx_scenario']=='Reported')& (result[''].notna())]

,year,month,fx_scenario,account_type,to_currency,from_currency,rate,rate_adj,quarter
72,2025,3.0,Reported,REVENUE,USD,TWD,0.047995,0.988777,1
73,2025,3.0,Reported,REVENUE,USD,EUR,0.754254,1.048347,1
74,2025,3.0,Reported,REVENUE,USD,ZAR,1.071192,1.014026,1
75,2025,3.0,Reported,COST,USD,TWD,0.254831,1.020559,1
76,2025,3.0,Reported,COST,USD,EUR,0.363022,1.021056,1
77,2025,3.0,Reported,COST,USD,ZAR,0.770946,1.020018,1
180,2025,9.0,Reported,REVENUE,USD,TWD,0.784913,0.966379,3
181,2025,9.0,Reported,REVENUE,USD,EUR,0.965050,0.950836,3
182,2025,9.0,Reported,REVENUE,USD,ZAR,0.679733,1.042361,3
183,2025,9.0,Reported,COST,USD,TWD,0.973574,0.983191,3


In [12]:
# 按quarter进行backward fill
print("="*80)
print("执行backward fill操作:")
print("="*80)

# 为每个组合进行backward fill
groupby_cols = ['year', 'fx_scenario', 'account_type', 'to_currency', 'from_currency']

# 先按组合和月份排序，确保数据顺序正确
result = result.sort_values(groupby_cols + ['month'])

# 对每个组合进行backward fill
result['rate_adj'] = result.groupby(groupby_cols)['rate_adj'].bfill()

print("Backward fill完成后的数据验证:")
print()

# 验证各scenario的数据分布
print("各scenario有rate_adj值的月份分布 (填充后):")
for scenario in fx_scenarios:
    data = result[
        (result['fx_scenario'] == scenario) & 
        (result['rate_adj'].notna())
    ]['month'].unique()
    print(f"{scenario}: {sorted(data) if len(data) > 0 else '无数据'}")

print("\n验证数据样本 (Reported scenario - 填充后):")
sample = result[
    (result['fx_scenario'] == 'Reported') &
    (result['account_type'] == 'REVENUE') &
    (result['to_currency'] == 'USD') &
    (result['from_currency'] == 'TWD') &
    (result['year'] == 2025)
].sort_values('month')[['year', 'month', 'quarter', 'rate_adj']]
print(sample)

print("\n验证数据样本 (Constant Currency YTD - Half Year - 填充后):")
sample2 = result[
    (result['fx_scenario'] == 'Constant Currency YTD - Half Year') &
    (result['account_type'] == 'REVENUE') &
    (result['to_currency'] == 'USD') &
    (result['from_currency'] == 'TWD') &
    (result['year'] == 2025)
].sort_values('month')[['year', 'month', 'quarter', 'rate_adj']]
print(sample2)

# 检查填充效果 - 显示按quarter的分布
print("\n按quarter查看填充效果:")
quarter_check = result[
    (result['fx_scenario'] == 'Reported') &
    (result['account_type'] == 'REVENUE') &
    (result['to_currency'] == 'USD') &
    (result['from_currency'] == 'TWD') &
    (result['year'] == 2025)
].groupby('quarter').agg({
    'rate_adj': ['count', 'first', lambda x: x.nunique()]
}).round(6)
quarter_check.columns = ['count', 'rate_adj_value', 'unique_values']
print(quarter_check)

执行backward fill操作:
Backward fill完成后的数据验证:

各scenario有rate_adj值的月份分布 (填充后):
Reported: [1.0, 2.0, 3.0, 7.0, 8.0, 9.0]
Constant Currency YTD - Half Year: 无数据
Constant Currency - YTD: 无数据
Constant Currency - Reporting: [1.0, 2.0, 3.0, 7.0, 8.0, 9.0]
Constant Currency - Reporting(-1): [1.0, 2.0, 3.0, 7.0, 8.0, 9.0]
Constant Currency - Reporting(-2): [1.0, 2.0, 3.0, 7.0, 8.0, 9.0]

验证数据样本 (Reported scenario - 填充后):
     year  month  quarter  rate_adj
0    2025    1.0        1  0.988777
36   2025    2.0        1  0.988777
72   2025    3.0        1  0.988777
108  2025    7.0        3  0.966379
144  2025    8.0        3  0.966379
180  2025    9.0        3  0.966379

验证数据样本 (Constant Currency YTD - Half Year - 填充后):
     year  month  quarter  rate_adj
6    2025    1.0        1       NaN
42   2025    2.0        1       NaN
78   2025    3.0        1       NaN
114  2025    7.0        3       NaN
150  2025    8.0        3       NaN
186  2025    9.0        3       NaN

按quarter查看填充效果:
         count 